In [ ]:
!wget http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip   

--2022-07-19 09:05:48--  http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Resolving infolab.tamu.edu (infolab.tamu.edu)... 45.55.217.29
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip [following]
--2022-07-19 09:05:49--  https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 263659672 (251M) [application/zip]
Saving to: ‘social_honeypot_icwsm_2011.zip’

social_honeypot_icw 100%[===================>] 251.45M  15.6MB/s    in 18s     

2022-07-19 09:06:08 (14.3 MB/s) - ‘social_honeypot_icwsm_2011.zip’ saved [263659672/263659672]



In [ ]:
!unzip social_honeypot_icwsm_2011.zip

Archive:  social_honeypot_icwsm_2011.zip
   creating: social_honeypot_icwsm_2011/
  inflating: social_honeypot_icwsm_2011/content_polluters.txt  
   creating: __MACOSX/
   creating: __MACOSX/social_honeypot_icwsm_2011/
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters.txt  
  inflating: social_honeypot_icwsm_2011/content_polluters_followings.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters_followings.txt  
  inflating: social_honeypot_icwsm_2011/content_polluters_tweets.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters_tweets.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._legitimate_users.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users_followings.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._legitimate_users_followings.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users_tweets.txt  
  inflating: __MACOSX/soci

In [ ]:
import pandas as pd
import numpy as np
import re, string
import operator
import networkx as nx
from numba import jit, cuda

import timeit

In [ ]:
!pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from operator import mul
import functools
from community import community_louvain
import community
import networkx as nx
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from scipy.stats import zscore

import matplotlib.pyplot as plt
from operator import itemgetter

In [ ]:
start = timeit.default_timer()

In [ ]:

def func(string):
   return pd.Series(string.split(',')).astype(int)
  
  
dfContent_polluters = pd.read_csv('social_honeypot_icwsm_2011/content_polluters.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"])

dfContent_polluters_followings = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)

dfContent_polluters_tweets = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)

dfLegitimate_users = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"])

dfLegitimate_users_followings = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)


dfLegitimate_users_tweets = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)

In [ ]:
def get_user_id(data_frame):
    df = pd.DataFrame({"UserID": data_frame["UserID"]})
    return df

In [ ]:
data_Users_polluters = get_user_id(dfContent_polluters)
data_Users_legitimate = get_user_id(dfLegitimate_users)

# data_Users_polluters.insert(0,"Class",1)
# data_Users_legitimate.insert(0,"Class",0)

data_set_users = pd.concat([data_Users_polluters, data_Users_legitimate])

print("data_set_users : ",len(data_set_users))

    # /////////////////////////////////////////////////////
    # print("dfLegitimate_users_followings : ",len(dfLegitimate_users_followings))
    # print("dfContent_polluters_followings : ",len(dfContent_polluters_followings))

dfLegitimate_users_followings = dfLegitimate_users_followings.head(100)
dfContent_polluters_followings = dfContent_polluters_followings.head(100)

data_set_users :  41499


In [ ]:
import tweepy as tweepy

In [ ]:
# authenticaing twitter api user
# consumer_key  = 'M4fmVhY4vBA4DtNxOIFvX6Eq8'
# consumer_secret = 'GugDeK1aWzN7fNszABakeNUIiHsb4CKkdrbucnPBOKVHxDu6WE'
# access_token = '4307562143-0jtF4CXWvVRmY4loDhXnoMSDwJBiYbjJLSAt1Az'
# access_token_secret = 'B88JPqU7qfQD3dBGpWIg7QGUJtLoHxZrJRqjNzzQqBLJs'

# consumer_key  = 'd9Ksoz6Wb1jD0mqbW8rjaSNb7'
# consumer_secret = 'pHXnVSJeLbOxaYlbOR7BWFdDNhZSF6IzegZV87qUSUqy6Qe8qG'
# access_token = '3648603434-dGRu1nHet22tdoYeqaAGoN8MyZrNw9oXZQvGZUD'
# access_token_secret = 'PZ8pcQBCb5zVPLRQNVQZc3Yzi0rz1wPef6O7RO7gzcvOf'

# consumer_key  = 'cyXcjxUc1ZlLD2v1DjaJrLiVI'
# consumer_secret = 'pD3mk041ROBiSSwt2C3kytR9eF1zYrX1vBD8whpDln5Zq4y1jX'
# access_token = '1279253779540410373-8ODCqefNcIvSFXSrwKH7DPhtQTda3i'
# access_token_secret = 'nVGU8WyEdGCQ2gU1qQ57NXxLpYQts4Emt1UPM6GVlYaav'


# consumer_key = 'BJZXO2O7qffWEWiBzztaI0pmz'
# consumer_secret = '1V78UxJL3ahxfeyRxchZX9OD5Se35NP0FoRVk2Kt1Q6arV1s9t'
# access_token = '1421415547607126016-Gt5QxcyBoPTQr1rKRRogxhgO46JdrW'
# access_token_secret = 'afPcI7OlTv9Vbm55ecSijEWhRIDbhPdfOnz2lnGF8qIal'

# mori account
consumer_key = 'Cq1sycJH2dda1jZCvMlqiBT7b'
consumer_secret = 'NAmkgtvnezW42U80ndOqeD8bfIZnM8KF8k9g9wl73SNa9mjfis'
access_token = '1544279124482703364-Taf0ZjnGuRW4HCfAidh4jhRK4xZ3RI'
access_token_secret = 'xtst9BPdI4AGs1itwmmXSII5J8V25D2vyRj7lKYdnqHKJ'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth , wait_on_rate_limit=True)


In [ ]:
def func(string):
   return pd.Series(string.split(',')).astype(int)
  

In [ ]:

def folowersSTD(user) :
    freiend_list = ''
    friends = api.friends(id=user)[:10]
    # print(len(friends))
    for friend in friends:
        freiend_list = freiend_list+str(friend.id)+','
    # print(func(freiend_list[:-1]))
    vari = func(freiend_list[:-1])
    return vari.std()

In [ ]:
from datetime import datetime
now = datetime.now()
 
print("now =", now)
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)	


now = 2022-07-19 09:07:16.891454
date and time = 19/07/2022 09:07:16


In [ ]:
def getTweets(user) :
    tweets = api.user_timeline(id=user, 
                           # 200 is the maximum allowed count
                           count=10,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    return tweets

In [ ]:
import time

In [ ]:
f_account = open('/content/drive/MyDrive/users_account2.txt', 'w') 

In [ ]:
notExistUser = []
existUser = []
for seed_id , SeriesOfNumberOfFollowings in dfContent_polluters_followings[:10].itertuples():
    print("***************** seed_id ", seed_id)
    for userId in SeriesOfNumberOfFollowings:
        try:
            friend=api.get_user(userId)
            print("***************** friend ", friend)
            # print(friend.screen_name)
            f_account.write(friend.screen_name)
            f_account.write('\t')
            # print(friend.id)
            f_account.write(str(friend.id))
            f_account.write('\t')
            # print(friend.created_at)
            f_account.write(str(friend.created_at))
            f_account.write('\t')
            # print(now)
            f_account.write(str(now))
            f_account.write('\t')
            # print(friend.friends_count)
            f_account.write(str(friend.friends_count))
            f_account.write('\t')
            # print(friend.followers_count)
            f_account.write(str(friend.followers_count))
            f_account.write('\t')
            # print(friend.statuses_count)
            f_account.write(str(friend.statuses_count))
            f_account.write('\t')
            # print(len(friend.screen_name))
            f_account.write(str(len(friend.screen_name)))
            f_account.write('\t')
            # print(folowersSTD(friend.id))
            f_account.write(str(folowersSTD(friend.id)))
            f_account.write('\t')
            f_account.write('\n')
            existUser.append(friend.id)
        except Exception as e: 
              print(e) 
              notExistUser.append(userId)
              time.sleep(10)
              pass

***************** seed_id  6301
***************** friend  User(_api=<tweepy.api.API object at 0x7fc0bc6ff9d0>, _json={'id': 3269, 'id_str': '3269', 'name': 'Jay', 'screen_name': 'jl2im', 'location': 'Chicago, IL', 'profile_location': None, 'description': '....', 'url': 'http://t.co/tw6zkvfblU', 'entities': {'url': {'urls': [{'url': 'http://t.co/tw6zkvfblU', 'expanded_url': 'https://twitter.com/jl2im', 'display_url': 'twitter.com/jl2im', 'indices': [0, 22]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 864, 'friends_count': 385, 'listed_count': 6, 'created_at': 'Thu Jul 27 20:24:21 +0000 2006', 'favourites_count': 21, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 1373, 'lang': None, 'status': {'created_at': 'Sat May 07 13:28:51 +0000 2022', 'id': 1522931469592760320, 'id_str': '1522931469592760320', 'text': '#Survivor #MonkeyRun https://t.co/Wxd5wqYFUb', 'truncated': False, 'entities': {'hashtags': [{'text': 'S

In [ ]:
f_account.close()

In [ ]:
f_tweet = open('/content/drive/MyDrive/users_tweets.txt', 'w') 

In [ ]:
for seed_id in existUser['UserID'][:10]:
    try:
        tweet = getTweets(seed_id.id)
        for info in tweet[:50]:
            f_tweet.write(str(seed_id.id))
            f_tweet.write('\t')
            # print("ID: {}".format(info.id))
            f_tweet.write(str(format(info.id)))
            f_tweet.write('\t')
            # print(info.created_at)
            f_tweet.write(str(info.created_at))
            f_tweet.write('\t')
            # print(info.full_text)
            f_tweet.write(info.full_text.replace("\n", ""))
            f_tweet.write('\t')
            f_tweet.write('\n')
    except Exception as e: 
          print(e) 
          time.sleep(180) 
          pass

In [ ]:

f_tweet.close()